In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras import layers , activations , models , preprocessing
from tensorflow.keras import preprocessing , utils
import re

In [2]:
lines = open('data/movie_lines.txt',  encoding='utf-8', errors ='ignore').read().split('\n')
conversations_lines = open('data/movie_conversations.txt',  encoding='utf-8', errors ='ignore').read().split('\n')

In [3]:
lines[:5]

['L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!',
 'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!',
 'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.',
 'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?',
 "L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go."]

In [4]:
conversations_lines[:5]

["u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']"]

In [5]:
# create a dictionary with key = id and value = text
id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ')
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]

In [6]:
# create a list of all conversations' lines
# lsit of lists
conversations = [ ]
for line in conversations_lines:
    _line = line.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
    conversations.append(_line.split(','))

In [7]:
conversations[:5]

[['L194', 'L195', 'L196', 'L197'],
 ['L198', 'L199'],
 ['L200', 'L201', 'L202', 'L203'],
 ['L204', 'L205', 'L206'],
 ['L207', 'L208']]

In [8]:
# sort the lines into inupt texts and output texts
inputs = []
outputs = []

for conversation in conversations:
    for i in range(len(conversation)-1):
        inputs.append(id2line[conversation[i]])
        outputs.append(id2line[conversation[i+1]])

In [9]:
# load a couple of sorted input-output texts
for i in range(5):
    print(inputs[i])
    print(outputs[i]+'\n')
    

Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.
Well, I thought we'd start with pronunciation, if that's okay with you.

Well, I thought we'd start with pronunciation, if that's okay with you.
Not the hacking and gagging and spitting part.  Please.

Not the hacking and gagging and spitting part.  Please.
Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?

You're asking me out.  That's so cute. What's your name again?
Forget it.

No, no, it's my fault -- we didn't have a proper introduction ---
Cameron.



In [10]:
print(len(inputs))
print(len(outputs))

221616
221616


In [11]:
# change word format and remove unnecessary characters
def clean_text(text):

    text = text.lower()
    
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    
    return text

In [12]:
# clean the data
clean_inputs = []
for text in inputs:
    clean_inputs.append(clean_text(text))
    
clean_outputs = []    
for text in outputs:
    clean_outputs.append(clean_text(text))

In [13]:
# load a couple of sorted input-output texts
for i in range(5):
    print(clean_inputs[i])
    print(clean_outputs[i]+'\n')

can we make this quick  roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad  again
well i thought we would start with pronunciation if that is okay with you

well i thought we would start with pronunciation if that is okay with you
not the hacking and gagging and spitting part  please

not the hacking and gagging and spitting part  please
okay then how about we try out some french cuisine  saturday  night

you are asking me out  that is so cute that is your name again
forget it

no no it is my fault  we did not have a proper introduction 
cameron



In [14]:
# find the length of sentences
lengths = []
for text in clean_inputs:
    lengths.append(len(text.split()))
for text in clean_outputs:
    lengths.append(len(text.split()))

# create a dataframe so that the values can be inspected
lengths = pd.DataFrame(lengths, columns=['counts'])

In [15]:
lengths.describe()

,counts
count,443232.000000
mean,10.872094
std,12.215895
min,0.000000
25%,4.000000
50%,7.000000
75%,14.000000
max,555.000000


In [16]:
print(np.percentile(lengths, 80))
print(np.percentile(lengths, 85))
print(np.percentile(lengths, 90))
print(np.percentile(lengths, 95))
print(np.percentile(lengths, 99))

16.0
19.0
24.0
32.0
58.0


In [17]:
# remove input and output texts that are shorter than 2 words and longer than 10 words.
min_line_length = 2
max_line_length = 10

# filter out the input texts that are too short/long
short_inputs_temp = []
short_outputs_temp = []

i = 0
for text in clean_inputs:
    if len(text.split()) >= min_line_length and len(text.split()) <= max_line_length:
        short_inputs_temp.append(text)
        short_outputs_temp.append(clean_outputs[i])
    i += 1

# filter out the output texts that are too short/long
short_inputs = []
short_outputs = []

i = 0
for text in short_outputs_temp:
    if len(text.split()) >= min_line_length and len(text.split()) <= max_line_length:
        short_outputs.append(text)
        short_inputs.append(short_inputs_temp[i])
    i += 1

In [18]:
# Compare the number of lines we will use with the total number of lines.
print('# of questions:', len(short_inputs))
print('# of answers:', len(short_outputs))
print('% of data used: {}%'.format(round(len(short_inputs)/len(inputs),2)*100))

# of questions: 74473
# of answers: 74473
% of data used: 34.0%


In [19]:
# create a dictionary for the frequency of the vocabulary
vocab = {}
for text in short_inputs:
    for word in text.split():
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1
            
for text in short_outputs:
    for word in text.split():
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1

In [20]:
# remove rare words from the vocabulary.
# aim to replace fewer than 5% of words with <UNK>
threshold = 10
count = 0
for k,v in vocab.items():
    if v >= threshold:
        count += 1

In [21]:
print("Size of total vocab:", len(vocab))
print("Size of vocab we will use:", count)

Size of total vocab: 27254
Size of vocab we will use: 3875


In [22]:
# in case we want to use a different vocabulary sizes for the source and target text, 
# set different threshold values.
# create dictionaries to provide a unique integer for each word.
inputs_vocab_to_int = {}

word_num = 0
for word, count in vocab.items():
    if count >= threshold:
        inputs_vocab_to_int[word] = word_num
        word_num += 1
        
outputs_vocab_to_int = {}

word_num = 0
for word, count in vocab.items():
    if count >= threshold:
        outputs_vocab_to_int[word] = word_num
        word_num += 1

In [23]:
# add the unique codes to the vocabulary dictionaries.
codes = ['<PAD>','<EOS>','<UNK>','<GO>']

for code in codes:
    inputs_vocab_to_int[code] = len(inputs_vocab_to_int)+1
    
for code in codes:
    outputs_vocab_to_int[code] = len(outputs_vocab_to_int)+1

In [24]:
# sanity check
print(outputs_vocab_to_int['<GO>'])
print(outputs_vocab_to_int['<UNK>'])
print(inputs_vocab_to_int['<GO>'])

3879
3878
3879


In [25]:
# create dictionaries to map the unique integers to their respective words.
# reverse of vocab_to_int
inputs_int_to_vocab = {i: v for v, i in inputs_vocab_to_int.items()}
outputs_int_to_vocab = {o: v for v, o in outputs_vocab_to_int.items()}

In [26]:
# the length of the dictionaries.
print(len(inputs_vocab_to_int))
print(len(inputs_int_to_vocab))
print(len(outputs_vocab_to_int))
print(len(outputs_int_to_vocab))

3879
3879
3879
3879


In [27]:
# add the end of sentence token to the end of every answer.
for i in range(len(short_outputs)):
    short_outputs[i] += ' <EOS>'

In [28]:
# convert the text to integers. 
# replace any words that are not in the vocabulary with <UNK> 
inputs_int = []
for text in short_inputs:
    ints = []
    for word in text.split():
        if word not in inputs_vocab_to_int:
            ints.append(inputs_vocab_to_int['<UNK>'])
        else:
            ints.append(inputs_vocab_to_int[word])
    inputs_int.append(ints)
    
outputs_int = []
for text in short_outputs:
    ints = []
    for word in text.split():
        if word not in outputs_vocab_to_int:
            ints.append(outputs_vocab_to_int['<UNK>'])
        else:
            ints.append(outputs_vocab_to_int[word])
    outputs_int.append(ints)

In [29]:
print(len(inputs_int))
print(len(outputs_int))

74473
74473


In [30]:
# calculate what percentage of all words have been replaced with <UNK>
word_count = 0
unk_count = 0

for text in inputs_int:
    for word in text:
        if word == inputs_vocab_to_int["<UNK>"]:
            unk_count += 1
        word_count += 1
    
for text in outputs_int:
    for word in text:
        if word == outputs_vocab_to_int["<UNK>"]:
            unk_count += 1
        word_count += 1
    
unk_ratio = round(unk_count/word_count,4)*100
    
print("Total number of words:", word_count)
print("Number of times <UNK> is used:", unk_count)
print("Percent of words that are <UNK>: {}%".format(round(unk_ratio,3)))

Total number of words: 887697
Number of times <UNK> is used: 54389
Percent of words that are <UNK>: 6.13%


In [31]:
# padding encoder_input_data
padded_inputs = preprocessing.sequence.pad_sequences(inputs_int , maxlen=max_line_length , padding='post')
encoder_input_data = np.array(padded_inputs)
print(encoder_input_data.shape , max_line_length)

(74473, 10) 10


In [32]:
# padding decoder_input_data
padded_outputs = preprocessing.sequence.pad_sequences(outputs_int , maxlen=max_line_length, padding='post')
decoder_input_data = np.array(padded_outputs)
print(decoder_input_data.shape, max_line_length)

(74473, 10) 10


In [40]:
# padding decoder_output_data afrer removing <GO>
for i in range(len(outputs_int)) :
    outputs_int[i] = outputs_int[i][:-1]
padded_outputs = preprocessing.sequence.pad_sequences(outputs_int, maxlen=max_line_length, padding='post')
onehot_answers = utils.to_categorical(padded_outputs, len(outputs_vocab_to_int))
decoder_output_data = np.array(onehot_answers)
print(decoder_output_data.shape)

(74473, 10, 3879)


In [45]:
decoder_output_data[1:5]

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]]], dtyp